In [1]:
import numpy as np
import pandas as pd
import torch
import json
import itertools
import scipy.stats

In [2]:
!pip install evaluate
!pip install bert_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 KB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 25.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 KB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 107.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 97.7 MB/s eta 0:00:00


In [3]:
!git clone https://github.com/niru-umass-dev/696ds_project1.git

Cloning into '696ds_project1'...
remote: Enumerating objects: 35, done.
remote: Counting objects: 100% (35/35), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 35 (delta 17), reused 24 (delta 10), pack-reused 0
Unpacking objects: 100% (35/35), 317.06 KiB | 467.00 KiB/s, done.


In [4]:
import bert_score
SCORER = bert_score.BERTScorer(model_type="microsoft/deberta-xlarge-mnli", lang="en", rescale_with_baseline=True)

Some weights of the model checkpoint at microsoft/deberta-xlarge-mnli were not used when initializing DebertaModel: ['classifier.weight', 'pooler.dense.weight', 'pooler.dense.bias', 'classifier.bias']
- This IS expected if you are initializing DebertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
data = json.load(open('696ds_project1/data/combined.json', 'r'))
dev = data['dev']
test = data['test']

In [99]:
def bert_score_ref_comm_a():

  matrix = []

  for i in itertools.chain(dev,test):

    inner = []

    for j in range(len(i['entity_a_summary'])):

      scores = SCORER.score([i['entity_a_summary'][j]], [i['gen_cont_a']])

      inner.append([i.detach().numpy()[0] for i in scores])


    matrix.append(list(np.mean(inner, axis = 0)))

  return matrix

def bert_score_ref_comm_b():

  matrix = []

  for i in itertools.chain(dev,test):

    inner = []

    for j in range(len(i['entity_b_summary'])):

      scores = SCORER.score([i['entity_b_summary'][j]], [i['gen_cont_b']])

      inner.append([i.detach().numpy()[0] for i in scores])


    matrix.append(list(np.mean(inner, axis = 0)))

  return matrix

In [98]:
ref_gen_scores_a = bert_score_ref_comm_a()
ref_gen_scores_a

[[0.29928556, 0.25497016, 0.2792441],
 [0.43448618, 0.27518177, 0.35296798],
 [0.27507168, 0.28064325, 0.28062353],
 [0.21812546, 0.25520954, 0.23922284],
 [0.3619239, 0.41619423, 0.39070424],
 [0.29052112, 0.25161996, 0.27357802],
 [0.2596512, 0.280409, 0.27285984],
 [0.37740675, 0.37256142, 0.37703028],
 [0.353544, 0.32091355, 0.3386462],
 [0.19770946, 0.26343143, 0.23256391],
 [0.33417678, 0.40272048, 0.37002754],
 [0.2907435, 0.32612646, 0.31087804],
 [0.3354497, 0.38182315, 0.3606912],
 [0.27121362, 0.34690884, 0.31078622],
 [0.24024369, 0.18936609, 0.21511126],
 [0.34467044, 0.4077817, 0.37789544],
 [0.26929143, 0.3505896, 0.31139755],
 [0.30460024, 0.33186877, 0.32051203],
 [0.13671279, 0.30856892, 0.21958072],
 [0.35326138, 0.24370559, 0.29888418],
 [0.2897054, 0.34722927, 0.32053995],
 [0.32074782, 0.3602376, 0.34224746],
 [0.2552064, 0.3130957, 0.28617725],
 [0.35081258, 0.37745205, 0.3662751],
 [0.23900557, 0.31965134, 0.2808384],
 [0.35068536, 0.39691448, 0.37578678],
 [0.3

In [100]:
ref_gen_scores_b = bert_score_ref_comm_b()
ref_gen_scores_b

[[0.2909541, 0.30147818, 0.29901266],
 [0.29019913, 0.2186964, 0.2564719],
 [0.30515817, 0.34088764, 0.3254026],
 [0.22976929, 0.14072402, 0.1868543],
 [0.28949153, 0.33865905, 0.31603336],
 [0.3552139, 0.39446864, 0.3770388],
 [0.18388271, 0.26351762, 0.22556408],
 [0.34787023, 0.35762987, 0.35529527],
 [0.36802018, 0.30937207, 0.34022593],
 [0.24413256, 0.25632226, 0.25320038],
 [0.3179465, 0.38259432, 0.35161912],
 [0.34733453, 0.3352562, 0.34299037],
 [0.30145922, 0.4476426, 0.37324294],
 [0.19755714, 0.17819685, 0.19019377],
 [0.3576088, 0.33225003, 0.34676227],
 [0.2966628, 0.30245975, 0.30177775],
 [0.29439637, 0.4263018, 0.35976925],
 [0.21964443, 0.35390624, 0.2862802],
 [0.3826348, 0.3692017, 0.37830353],
 [0.2658631, 0.10538915, 0.18231456],
 [0.30986726, 0.31885755, 0.31709388],
 [0.26326454, 0.334034, 0.30049896],
 [0.36376223, 0.28882372, 0.32738563],
 [0.29097652, 0.3286593, 0.31160358],
 [0.3644174, 0.38182488, 0.37510502],
 [0.28032187, 0.3185725, 0.30192354],
 [0.2490

# BERTScore

In [6]:
# compute Bertscore on contrastive reference summaries a and b
def bert_score_ref(entity_a_summary, entity_b_summary):

  matrix = []

  for i in itertools.chain(dev,test):

    inner = []

    for j in range(len(i[entity_a_summary])):

      scores = SCORER.score([i[entity_a_summary][j]], [i[entity_b_summary][j]])

      inner.append([i.detach().numpy()[0] for i in scores])


    matrix.append(list(np.mean(inner, axis = 0)))

  return matrix

# compute Bertscore on contrastive generated summaries a and b
def bert_score_gen(gen_cont_a, gen_cont_b):

  matrix = []

  for i in itertools.chain(dev,test):


    scores = SCORER.score([i[gen_cont_a]], [i[gen_cont_b]])
    matrix.append([i.detach().numpy()[0] for i in scores])

    
  return matrix

In [9]:
cont_ref_scores = bert_score_ref('entity_a_summary', 'entity_b_summary')
cont_gen_scores = bert_score_gen('gen_cont_a', 'gen_cont_b')

comm_ref_a = bert_score_ref('entity_a_summary' , 'common_summary')
comm_gen_a = bert_score_gen('gen_cont_a', 'gen_comm_a')

comm_ref_b = bert_score_ref('entity_b_summary' , 'common_summary')
comm_gen_b = bert_score_gen('gen_cont_b', 'gen_comm_b')

# BERTScore Aggregations

In [13]:
print('BERTScore mean reference contrastive summaries: %.4f' % np.mean(np.array(cont_ref_scores)[:,2]))
print('BERTScore mean generated contrastive summaries: %.4f' % np.mean(np.array(cont_gen_scores)[:,2]))

print('BERTScore mean reference contrastive A with common: %.4f' % np.mean(np.array(comm_ref_a)[:,2]))
print('BERTScore mean generated contrastive A with common: %.4f' % np.mean(np.array(comm_gen_a)[:,2]))

print('BERTScore mean reference contrastive B with common: %.4f' % np.mean(np.array(comm_ref_b)[:,2]))
print('BERTScore mean generated contrastive B with common: %.4f' % np.mean(np.array(comm_gen_b)[:,2]))


BERTScore mean reference contrastive summaries: 0.2654
BERTScore mean generated contrastive summaries: 0.2691
BERTScore mean reference contrastive A with common: 0.2046
BERTScore mean generated contrastive A with common: 0.2975
BERTScore mean reference contrastive B with common: 0.2141
BERTScore mean generated contrastive B with common: 0.2741


In [27]:
print('BERTScore mean all reference combinations: %.4f' %np.mean(np.hstack((np.array(cont_ref_scores)[:,2],np.array(comm_ref_a)[:,2],np.array(comm_ref_b)[:,2]))) )
print('BERTScore mean all generated combinations: %.4f' %np.mean(np.hstack((np.array(cont_gen_scores)[:,2],np.array(comm_gen_a)[:,2],np.array(comm_gen_b)[:,2]))) )

BERTScore mean all reference combinations: 0.2280
BERTScore mean all generated combinations: 0.2802


# Correlation

In [90]:
# Correlation between Contrastive Summaries
f1_ref = np.array(cont_ref_scores)[:,2]
f1_gen = np.array(cont_gen_scores)[:,2]
corr, _ = scipy.stats.pearsonr(f1_ref,f1_gen)
spear, _ = scipy.stats.spearmanr(f1_ref,f1_gen)
tau, _ = scipy.stats.kendalltau(f1_ref,f1_gen)
print('Pearsons correlation: %.4f' % corr)
print('Spearman Rho: %.4f' % spear)
print('Kendall’s Tau: %.4f' % tau)


Pearsons correlation: 0.1361
Spearman Rho: 0.1598
Kendall’s Tau: 0.1217


In [93]:
# Correlation between contrastive A and common summaries
f1_ref_comm_a = np.array(comm_ref_a)[:, 2]
f1_gen_comm_a = np.array(comm_gen_a)[:,2]
corr, _ = scipy.stats.pearsonr(f1_ref_comm_a,f1_gen_comm_a)
spear, _ = scipy.stats.spearmanr(f1_ref_comm_a,f1_gen_comm_a)
tau, _ = scipy.stats.kendalltau(f1_ref_comm_a,f1_gen_comm_a)
print('Pearsons correlation: %.4f' % corr)
print('Spearman Rho: %.4f' % spear)
print('Kendall’s Tau: %.4f' % tau)

Pearsons correlation: -0.1534
Spearman Rho: -0.0657
Kendall’s Tau: -0.0423


In [92]:
# Correlation between contrastive B and common summaries
f1_ref_comm_b = np.array(comm_ref_b)[:, 2]
f1_gen_comm_b = np.array(comm_gen_b)[:,2]
corr, _ = scipy.stats.pearsonr(f1_ref_comm_b,f1_gen_comm_b)
spear, _ = scipy.stats.spearmanr(f1_ref_comm_b,f1_gen_comm_b)
tau, _ = scipy.stats.kendalltau(f1_ref_comm_b,f1_gen_comm_b)
print('Pearsons correlation: %.4f' % corr)
print('Spearman Rho: %.4f' % spear)
print('Kendall’s Tau: %.4f' % tau)

Pearsons correlation: 0.0547
Spearman Rho: 0.1554
Kendall’s Tau: 0.1058
